In [10]:
import pandas as pd
import os
import glob
import time
from numba import jit

In [11]:
chunk_size = 500000
sorted_values = pd.DataFrame

In [13]:
#Sorts the values
def sortinize():  
    sorted_values = pd.read_csv('2022_place_canvas_history.csv.gzip', compression='gzip', engine="python")
    sorted_values = sorted_values.sort_values(by=(['user_id']), kind='heapsort')
    print(sorted_values)
start_time = time.time()

sortinize()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#Splits up the dataset into chunks of 500,000 rows each
@jit(nopython=True)
def chunkinize():
   for i,chunk in enumerate(sorted_values):
      chunk.to_csv('./tmp/split_csv_pandas/chunk{}.csv.gzip'.format(i), index=False, compression='gzip')
   print('The chunks have been written to ./tmp/split_csv_pandas')

start_time = time.time()
chunkinize()
print("--- %s seconds ---" % (time.time() - start_time))

In [2]:
#Counts how many rows is in the dataset and prints it, must be run for the rest to work
path = './tmp/split_csv_pandas'
csv_files = glob.glob(os.path.join(path, "*.csv"))

sum=0
for i, df_chunk in enumerate(csv_files):
    sum += chunk_size
sum -= chunk_size

last_path = f'./tmp/split_csv_pandas/chunk{i}.csv'
last_len = pd.read_csv(last_path).shape[0]
sum += last_len

print('This dataset is: ', str(sum), ' rows long')

This dataset is:  160353104  rows long


The code cells under will generate a csv file with user_id and occurences


In [ ]:
#Loops throuh the entire dataset and count how many occurences of a specific user_id

for i, df_chunk in enumerate(csv_files):
    df_temp = pd.read_csv(df_chunk)

    freq = df_temp.groupby(['user_id']).size()
    freq.to_csv('./tmp/user_list_split_csv/chunk{}.csv.gzip'.format(i), index=True, compression='gzip')


In [ ]:
#Combines every user_list_csv chunk into one dataframe, Loooooooooooooooooooong runtime
path_user_list = './tmp/user_list_split_csv.gzip'
csv_files = glob.glob(os.path.join(path, "*.csv"))

len_sum = 0

for i, df_user in enumerate(csv_files):
    user_list = pd.read_csv('./results/user_list.csv', sep=',', encoding='utf-8', compression='gzip')
    user_list_temp = pd.read_csv(df_user, sep=',', encoding='utf-8', compression='gzip')
    len_sum += user_list_temp.shape[0]

    user_list = pd.merge(user_list, user_list_temp, on="user_id", compression='gzip')
    user_list.to_csv('./results/user_list.csv', index=True, compression='gzip')

print('The dataset contains ', str(len_sum), ' unique user_id s')

In [ ]:
#Writes the huuuge dataframe to a csv file


Now i will do the same for coordinates

In [ ]:
#Loops throuh the entire dataset and count how many occurences of a specific coordinate
for i, df_chunk in enumerate(csv_files):
    df_temp = pd.read_csv(df_chunk)

    freq = df_temp.groupby(['coordinate']).size()
    freq.to_csv('./tmp/coordinate_list_split_csv/chunk{}.csv'.format(i), index=True)

In [ ]:
#Combines every coordinate_csv chunk into one file, Loooooooooooooooooooong runtime
path_coordinate_list = './tmp/coordinate_list_split_csv'
csv_files = glob.glob(os.path.join(path, "*.csv"))

len_sum = 0
coordinate = pd.DataFrame()
for i, df_user in enumerate(csv_files):
    coordinate_temp = pd.read_csv(df_user, sep=',', encoding='utf-8')
    len_sum += coordinate_temp.shape[0]
    coordinate = pd.concat([coordinate, freq])
    print('This is ', i)
print('The dataset contains ', str(len_sum), ' unique coordinate s')

In [ ]:
#Skriver ut chunkene til en csv fil, kinda pointless :/
df = pd.concat((chunk for chunk in pd.read_csv('2022_place_canvas_history.csv',chunksize=1000000)))
df.to_csv('./results/out.csv', sep=',', encoding='utf-8', index=True)